In [ ]:
from netCDF4 import Dataset, num2date
import numpy as np
import pandas as pd
import xarray as xr
import os
import time
import matplotlib.pyplot as plt
import matplotlib
import gc
from matplotlib import transforms
import matplotlib.colors as clrs
import glob

In [ ]:
for i in np.arange(0,12):
    filename = "/gpfs/fs7/dfo/hpcmc/comda/stod000/DATA/WOD/O2/oxy_wod_monthly_"+str(i+1)+"_CREG025.nc"
    data = Dataset(filename, "r", format="NETCDF4")
    X = data.variables['Oxygen'][:]
    if i==0:
        [nz,ny,nx]=np.shape(X)
        obs=np.zeros((12,nz,ny,nx))
        obsO2=np.transpose(obs,(0,3,1,2))
        print(np.shape(obsO2))
        #[nz,ny,nx]=np.shape(X)
        [nt,nz,ny,nx] = np.shape(obsO2)
    obsO2[i,:,:,:]=np.transpose(X,(2,0,1))

print(type(obsO2))
print(np.shape(obsO2))
print(np.nanmin(obsO2), np.nanmax(obsO2), np.nanmean(obsO2))

In [ ]:
data_file = Dataset("/gpfs/fs7/dfo/hpcmc/comda/stod000/CREG25/regions_mask_6.nc", "r", format="NETCDF4")
regindx = (data_file['regions_mask'][:])
print(regindx.shape)

data_file = Dataset("/home/fid000/WORK7/ANALYSIS/DATA/CREG025_mesh_mask.nc", "r", format="NETCDF4")
tmask = (data_file['tmask'][0,:,:,:])
print(tmask.shape)

print(np.nanmin(regindx),np.nanmean(regindx),np.nanmax(regindx))
print(np.min(tmask),np.mean(tmask),np.max(tmask))

tmask0=tmask[0,:,:].squeeze()

nxy=np.size(regindx)
I = np.reshape(regindx, (nxy,))
nreg=int(max(I))
mask = regindx
mask = mask.flatten()

In [ ]:
import itertools
fig, axes = plt.subplots(num=1, clear=True, nrows=3, ncols=3, figsize=(15, 15))

months=['September','October','November','December','January','February','March','April','May','June','July','August']
regions = ['Eastern Subarctic Pacific', 'Western Subarctic Pacific', 'Bering Sea', 'Beaufort Sea', 'Chukchi Sea', 'Baffin Bay (north)', 'Labrador Sea / Davis Strait', 'Atlantic/Arctic Transition', 'North Atlantic']
iz = np.arange(25,34)

variants = ['CREG025_LIM3_CANOE-VJC014l','CREG025_LIM3_CANOE-VJC014t','CREG025_LIM3_CANOE-VJC014w',
            'CREG025_LIM3_CANOE-VJC014x','CREG025_LIM3_CANOE-VJC014m','CREG025_LIM3_CANOE-VJC014n',
            'CREG025_LIM3_CANOE-VJC014o','CREG025_LIM3_CANOE-VJC014p','CREG025_LIM3_CANOE-VJC014y',
            'CREG025_LIM3_CANOE-VJC014q','CREG025_LIM3_CANOE-VJC014z','CREG025_LIM3_CANOE-VJC014r',
            'CREG025_LIM3_CANOE-VJC014s','CREG025_LIM3_CANOE-VJC014u','CREG025_LIM3_CANOE-VJC014v',
            'CREG025_LIM3_CANOE-VJC015a','CREG025_LIM3_CANOE-VJC015b','CREG025_LIM3_CANOE-VJC015c',
            'CREG025_LIM3_CANOE-VJC015d','CREG025_LIM3_CANOE-VJC015e','CREG025_LIM3_CANOE-VJC015f',
            'CREG025_LIM3_CANOE-VJC015g','CREG025_LIM3_CANOE-VJC015i','CREG025_LIM3_CANOE-VJC015j',
            'CREG025_LIM3_CANOE-VJC015k','CREG025_LIM3_CANOE-VJC015l','CREG025_LIM3_CANOE-VJC015m',
            'CREG025_LIM3_CANOE-VJC015n','CREG025_LIM3_CANOE-VJC015p','CREG025_LIM3_CANOE-VJC015q',
            'CREG025_LIM3_CANOE-VJC015r','CREG025_LIM3_CANOE-VJC015t','CREG025_LIM3_CANOE-VJC015u',
            'CREG025_LIM3_CANOE-VJC015v', 'CREG025_LIM3_CANOE-VJC015w','CREG025_LIM3_CANOE-VJCnep01',
            'CREG025_LIM3_CANOE-VJCsed01','CREG025_LIM3_CANOE-VJCsed02','CREG025_LIM3_CANOE-VJCsed03']




for asuff in variants:
    fig, axes = plt.subplots(num=1, clear=True, nrows=3, ncols=3, figsize=(15, 15))
    runid = str(asuff)
    data_all=np.zeros((12,nz,ny,nx))
    fig.suptitle('Oxygen_WOD_deep_'+asuff, fontsize=24)
    flist = glob.glob("/home/fid000/WORK7/ANALYSIS/model_evaluation/MonthlyMeans/O2/"+runid+"_monthly_O2.nc")
    flist.sort()
    ifile = 0
    for i, c, in enumerate(regions):
        cx=plt.subplot(3,3,i+1)
        colors =itertools.cycle(plt.get_cmap('viridis')(np.linspace(0,1, 12))) # potential pallettes: hsv, jet, gnuplot
        
        for im in np.arange(9,12):
            if im >= 0:
                filein = flist[ifile]
                data = Dataset(filein, "r", format="NETCDF4")
                modO2 = data.variables['O2'][:]
                tmaskz=tmask[iz,:,:].flatten()   
                ZZ=(np.zeros((10,ny,nx))+regindx).flatten()    # broadcast regions map into a 3D array with same data for each depth

                XOBS=obsO2[im,iz,:,:].squeeze().flatten()
                XMOD=modO2[im,iz,:,:].squeeze().flatten()
                ii=np.where( ~np.isnan(XOBS) )[0]
                XOBS=XOBS[ii]; XMOD=XMOD[ii]; ZZ=ZZ[ii]; tmaskz=tmaskz[ii];

                jj=np.where( (XOBS>0.) & (ZZ==(i+1)) & (tmaskz==1) )[0]
                colour = next(colors) 
                plt.plot(XOBS[jj], XMOD[jj], "o", color=colour, markeredgecolor=colour, label=im)
            if i > 5:
                plt.xlabel("Observations", fontsize=18)
            plt.ylabel("Model", fontsize=18)
            plt.title(c, fontsize=18)
            cx.tick_params(axis='both', labelsize=16)
            cx.set_aspect(1./cx.get_data_ratio())
        for im in np.arange(0,9):
            if im >= 0:
                filein = flist[ifile]
                print(filein)
                data = Dataset(filein, "r", format="NETCDF4")
                modO2 = data.variables['O2'][:]
                tmaskz=tmask[iz,:,:].flatten()   
                ZZ=(np.zeros((10,ny,nx))+regindx).flatten()    # broadcast regions map into a 3D array with same data for each depth

                XOBS=obsO2[im,iz,:,:].squeeze().flatten()
                XMOD=modO2[im,iz,:,:].squeeze().flatten()
                ii=np.where( ~np.isnan(XOBS) )[0]
                XOBS=XOBS[ii]; XMOD=XMOD[ii]; ZZ=ZZ[ii]; tmaskz=tmaskz[ii];

                jj=np.where( (XOBS>0.) & (ZZ==(i+1)) & (tmaskz==1) )[0]
                colour = next(colors) 
                plt.plot(XOBS[jj], XMOD[jj], "o", color=colour, markeredgecolor=colour, label=im)

            if i > 5:
                plt.xlabel("Observations", fontsize=18)

            plt.ylabel("Model", fontsize=18)
            plt.title(c, fontsize=18)
            plt.xlim([0, 400]); plt.ylim([0,400])
            cx.tick_params(axis='both', labelsize=16)
            cx.set_aspect(1./cx.get_data_ratio())
        
        XOBS=obsO2[:,iz,:,:].squeeze().flatten()
        XMOD=modO2[:,iz,:,:].squeeze().flatten()
        ZZ=(np.zeros((12,nz,ny,nx))+regindx).flatten()
        tmask12=(np.zeros((12,nz,ny,nx))+tmask0).flatten()
        ii=np.where( ~np.isnan(XOBS) )[0]
        
        XOBS=XOBS[ii]; XMOD=XMOD[ii]; ZZ=ZZ[ii]; tmask12=tmask12[ii];
        jj=np.where( (XOBS>0.) & (ZZ==(i+1)) & (tmask12==1) )[0]
        
        #r = np.corrcoef(XOBS[jj],XMOD[jj]); r = r[0,1]; '{:.2f}'.format(r); r = str(r); print(r)
        #r = np.corrcoef(XOBS[jj],XMOD[jj]); r = r[0,1]; r = "{:.2f}".format(r); r = str(r); print(r)

        #plt.text(.05, .8, s = 'r = '+r, transform=cx.transAxes, fontsize = 'xx-large')
        
        cx.plot([0, 1], [0, 1], color='grey', transform=cx.transAxes)
        if i == 1:
            plt.legend(labels=['September','October','November','December','January','February','March','April','May','June','July','August'],
                       fontsize='medium')
    
    plt.tight_layout(rect=(0,0,1,0.95), w_pad=0)

    plt.show()
    fig.savefig("/home/fid000/WORK7/ANALYSIS/model_evaluation/FIGURES/Books/O2_WOD_deep/O2_deep_WOD_"+runid)
    
    #fig.savefig("/home/fid000/WORK7/ANALYSIS/model_evaluation/FIGURES/Books/O2_Deep/O2_100_300m_"+runid)
#plt.close(fig)    

In [ ]:
# July

#fig, axes = plt.subplots(num=1, clear=True, nrows=6, ncols=5, figsize=(30, 20))
regions = ['Eastern Subarctic Pacific', 'Western Subarctic Pacific', 'Bering Sea', 'Beaufort Sea', 'Chukchi Sea', 'Baffin Bay (north)', 'Labrador Sea / Davis Strait', 'Atlantic/Arctic Transition', 'North Atlantic']
import itertools
iz = np.arange(25,34) #z level 25 is 108.03m and z level 34 is 300.88m

months=['January','February','March','April','May','June','July','August','September','October','November','December']
regions = ['Eastern Subarctic Pacific', 'Western Subarctic Pacific', 'Bering Sea', 'Beaufort Sea', 'Chukchi Sea', 'Baffin Bay (north)', 'Labrador Sea / Davis Strait', 'Atlantic/Arctic Transition', 'North Atlantic']
variants = ['VJC014l','VJC014w','VJC014p',
            'VJC014y','VJC014q','VJC014z',
            'VJC014r','VJC014s','VJC014u',
            'VJC014v','VJC015a','VJC015b',
            'VJC015c','VJC015d','VJC015e',
            'VJC015f','VJC015g','VJC015i',
            'VJC015j','VJC015k','VJC015l',
            'VJC015m','VJC015n','VJC015r',
            'VJC015oi','VJC015ol','VJC015om',
            'VJCnep01','VJCsed01','VJCsed03']


for im, mmon in enumerate(months):
    
    fig, axes = plt.subplots(num=1, clear=True, nrows=5, ncols=6, figsize=(50, 40))
    fig.suptitle('Mean O2 WOD (100_300m) by Region '+mmon, fontsize=34) 
    
    for a, asuff in enumerate(variants):
        cx=plt.subplot(5,6, a+1)
        runid = str('CREG025_LIM3_CANOE-'+asuff)
        print(runid)
        data_all=np.zeros((12,nz,ny,nx))
        flist = glob.glob("/home/fid000/WORK7/ANALYSIS/model_evaluation/MonthlyMeans/O2/"+runid+"_monthly_O2.nc")
        flist.sort()
        print(flist)
        ifile = 0
        colors =itertools.cycle(plt.get_cmap('viridis')(np.linspace(0,1, 9))) # potential pallettes: hsv, jet, gnuplot
            
        for i, c in enumerate(regions): 
            
            if i >= 0:
                
                #ifile=ifile+1
                print(ifile)
                filein = flist[ifile]
                #print(filein)
                data = Dataset(filein, "r", format="NETCDF4")
                modO2 = data.variables['O2'][:]

                tmaskz=tmask[iz,:,:].flatten()
                ZZ=(np.zeros((10,ny,nx))+regindx).flatten()    # broadcast regions map into a 3D array with same data for each depth
                XOBS=obsO2[im,iz,:,:].squeeze().flatten()
                XMOD=modO2[im,iz,:,:].squeeze().flatten()
                ii=np.where( ~np.isnan(XOBS) )[0]
                XOBS=XOBS[ii]; XMOD=XMOD[ii]; ZZ=ZZ[ii]; tmaskz=tmaskz[ii];
                jj=np.where( (XOBS>0.) & (ZZ==(i+1)) & (tmaskz==1) )[0]                          
              
                XOBS=np.mean(XOBS[jj]); XMOD=np.mean(XMOD[jj])
                
                colour = next(colors) 
                plt.plot(XOBS, XMOD, "o", color=colour, markeredgecolor=colour, markersize=15, label=regions)
                print(filein)

            #if i > 5:
            #    plt.xlabel("Observations", fontsize=18)
            plt.ylabel("Model", fontsize=28)
            plt.title(asuff, fontsize=28)
            plt.xlim([200, 450]); plt.ylim([200,450])
            cx.tick_params(axis='both', labelsize=26)
            cx.set_aspect(1./cx.get_data_ratio())
            if a ==1:
                plt.legend(labels=['Eastern Subarctic Pacific', 'Western Subarctic Pacific', 'Bering Sea', 'Beaufort Sea', 'Chukchi Sea', 'Baffin Bay (north)', 'Labrador Sea / Davis Strait', 'Atlantic/Arctic Transition', 'North Atlantic'],
                           fontsize='large')
        cx.plot([0, 1], [0, 1], color='grey', transform=cx.transAxes)
        
            
    plt.tight_layout(rect=(0,0,1,0.95), w_pad=0)
    plt.show()
    fig.savefig("/home/fid000/WORK7/ANALYSIS/model_evaluation/FIGURES/Means/O2_WOD_deep/O2_WOD_deep_month"+str("{:02d}".format(im+1)))